Now import all necessary libraries

In [7]:
import math

import cv2
import numpy as np
import glob

Variable debug to see intermediate results

In [8]:
debug = False

Now we import camera matrix and distortion coefficient

In [9]:
from matplotlib.pyplot import draw

K = np.load ( "framesVideoCalibration/uncalibrated/K.npy" )
dist = np.load ( "framesVideoCalibration/uncalibrated/dist.npy" )

listImages = glob.glob ( "framesVideo1/uncalibrated/*.jpg" )

# for each image in the list
for filename in listImages :

	# read the actual image
	img = cv2.imread ( filename , 0 )

	cv2.imwrite ( filename.replace ( "uncalibrated" , "backgroundremoved" ) , img )

Now we start to elaborate the video

In [10]:
# get list of frames of video 1
"""listImages = glob.glob ( "framesVideo1/uncalibrated/*.jpg" )

# for each image in the list
for filename in listImages :

	# read the actual image
	img = cv2.imread ( filename , 0 )

	cv2.imwrite ( filename.replace ( "uncalibrated" , "backgroundremoved" ) , equ )"""



'listImages = glob.glob ( "framesVideo1/uncalibrated/*.jpg" )\n\n# for each image in the list\nfor filename in listImages :\n\n\t# read the actual image\n\timg = cv2.imread ( filename , 0 )\n\n\tcv2.imwrite ( filename.replace ( "uncalibrated" , "backgroundremoved" ) , equ )'

In [11]:
import numpy as np
import cv2
import glob

# get list of frames of video 1
listImages = glob.glob ( "framesVideo1/uncalibrated/*.jpg" )

# for each image
for image in listImages :

	# open the actual image in color mode
	actual_image = cv2.imread ( image , cv2.IMREAD_COLOR )

	# open the actual image in gray mode
	gray_actual_image = cv2.imread ( image , cv2.IMREAD_GRAYSCALE )

	# make binary threshold on gray actual image
	_ , threshold_gray_actual_image = cv2.threshold ( gray_actual_image , 190 , 255 , cv2.THRESH_BINARY )

	# detect shape in threshold gray actual image using regions with same colors and intensity
	contours , _ = cv2.findContours ( threshold_gray_actual_image , cv2.RETR_TREE , cv2.CHAIN_APPROX_SIMPLE )

	# for each contour
	for contour in contours :

		# calculate the area of the actual region
		area_actual_region = cv2.contourArea ( contour )

		# if the area is acceptable
		if area_actual_region > 200 :

			# approximate the shape of the actual polygon
			approx = cv2.approxPolyDP ( contour , 0.01 * cv2.arcLength ( contour , True ) , True )

			# if we have 5 edges in the actual region
			if len ( approx ) == 5 :

				# we can dra contours of the actual region but in the colored image
				cv2.drawContours ( actual_image , [ approx ] , 0 , (0 , 0 , 255) , 2 )

	# save the image in the correct folder
	cv2.imwrite ( image.replace ( "uncalibrated" , "polygonborders" ) , actual_image )
	cv2.imwrite ( image.replace ( "uncalibrated" , "binarythreshold" ) , threshold_gray_actual_image )

In [43]:
import numpy as np
import cv2
import glob
import math

# get list of frames of video 1
listImages = glob.glob ( "framesVideo1/uncalibrated/*.jpg" )

# for each image
for image in listImages :

	# open the actual image in color mode
	actual_image = cv2.imread ( image , cv2.IMREAD_COLOR )

	# open the actual image in gray mode
	gray_actual_image = cv2.imread ( image , cv2.IMREAD_GRAYSCALE )

	# make binary threshold on gray actual image
	_ , threshold_gray_actual_image = cv2.threshold ( gray_actual_image , 190 , 255 , cv2.THRESH_BINARY )

	# save the threshold gray actual image
	cv2.imwrite ( image.replace ( "uncalibrated" , "binarythreshold" ) , threshold_gray_actual_image )

	# detect shape in threshold gray actual image using regions with same colors and intensity
	contours , hierarchy = cv2.findContours ( image = threshold_gray_actual_image , mode = cv2.RETR_TREE , method = cv2.CHAIN_APPROX_NONE )
	#print ( contours [ 164 ] )
	#cv2.drawContours(image=actual_image, contours=contours, contourIdx=-1, color=(0, 255, 0), thickness=2, lineType=cv2.LINE_AA)

	#save the index of actual contour
	index_actual_contour = 0

	# for each contour in threshold_gray_actual_image
	for contour in contours :

		# calculate the area of the actual region
		area_actual_region = cv2.contourArea ( contour )

		# approximate the shape of the actual polygon
		approx = cv2.approxPolyDP ( contour , 0.01 * cv2.arcLength ( contour , True ) , True )

		# if the area of the actual region is bigger than a minimum value
		if area_actual_region > 100 :

			# if we have 5 edges in the actual region
			if len ( approx ) == 5 :

				# calculate the perimeter of the contour
				perimeter = cv2.arcLength ( approx , True )

				# calculate the average of one edge
				avg_perimeter = perimeter / len ( approx )

				# save index of concave vertex
				first_edge_concave_vertex_clockwise = 0

				# for each edge
				for i in range ( 0 , len ( approx ) ) :

					# calculate the distance between the previous and the next one
					previous = math.dist ( [ approx [ i ] [ 0 ] [ 0 ] , approx [ i ] [ 0 ] [ 1 ] ] , [ approx [ (i - 1) % len ( approx ) ] [ 0 ] [ 0 ] , approx [ (i - 1) % len ( approx ) ] [ 0 ] [ 1 ] ] )
					next = math.dist ( [ approx [ i ] [ 0 ] [ 0 ] , approx [ i ] [ 0 ] [ 1 ] ] , [ approx [ (i + 1) % len ( approx ) ] [ 0 ] [ 0 ] , approx [ (i + 1) % len ( approx ) ] [ 0 ] [ 1 ] ] )

					# if they are both less than average edge
					if previous < avg_perimeter and next < avg_perimeter :

						# this is the concave vertex
						first_edge_concave_vertex_clockwise = i

						# calculate the edge of the outer polygon
						second = approx [ (i + 2) % len ( approx ) ] [ 0 ]
						third = approx [ (i + 3) % len ( approx ) ] [ 0 ]

						# calculate the middle point of the previous edge
						half_second_third = [ (second [ 0 ] + third [ 0 ]) // 2 , (second [ 1 ] + third [ 1 ]) // 2 ]

						# calculate the middle point between the previous middle point and the concave vertex
						half_point_polygon = [ (approx [ first_edge_concave_vertex_clockwise ] [ 0 ] [ 0 ] + half_second_third [ 0 ]) // 2 , (approx [ first_edge_concave_vertex_clockwise ] [ 0 ] [ 1 ] + half_second_third [ 1 ]) // 2 ]

						# draw a circle in the previous middle point
						#cv2.circle ( actual_image , (half_point_polygon [ 0 ] , half_point_polygon [ 1 ]) , 1 , (0 , 0 , 255) , -1 )

						# now draw circles of different color from the concave vertex in clockwise
						for j in range ( 0 , len ( approx ) ) :

							# if it is the concave vertex
							if j == 0 :

								# red circle
								actual_image = cv2.circle ( actual_image , (int ( approx [ (first_edge_concave_vertex_clockwise + j) % len ( approx ) ] [ 0 ] [ 0 ] ) , int ( approx [ (first_edge_concave_vertex_clockwise + j) % len ( approx ) ] [ 0 ] [ 1 ] )) , radius = 4 , color = (0 , 0 , 255) , thickness = -1 )

							# one vertex after the concave vertex
							elif j == 1 :

								# green circle
								actual_image = cv2.circle ( actual_image , (int ( approx [ (first_edge_concave_vertex_clockwise + j) % len ( approx ) ] [ 0 ] [ 0 ] ) , int ( approx [ (first_edge_concave_vertex_clockwise + j) % len ( approx ) ] [ 0 ] [ 1 ] )) , radius = 4 , color = (0 , 255 , 0) , thickness = -1 )

							# two vertex after the concave vertex
							elif j == 2 :

								# blue circle
								actual_image = cv2.circle ( actual_image , (int ( approx [ (first_edge_concave_vertex_clockwise + j) % len ( approx ) ] [ 0 ] [ 0 ] ) , int ( approx [ (first_edge_concave_vertex_clockwise + j) % len ( approx ) ] [ 0 ] [ 1 ] )) , radius = 4 , color = (255 , 0 , 0) , thickness = -1 )

							# three vertex after the concave vertex
							elif j == 3 :

								# yellow circle
								actual_image = cv2.circle ( actual_image , (int ( approx [ (first_edge_concave_vertex_clockwise + j) % len ( approx ) ] [ 0 ] [ 0 ] ) , int ( approx [ (first_edge_concave_vertex_clockwise + j) % len ( approx ) ] [ 0 ] [ 1 ] )) , radius = 4 , color = (0 , 255 , 255) , thickness = -1 )

							# four vertex after the concave vertex
							elif j == 4 :

								# pink circle
								actual_image = cv2.circle ( actual_image , (int ( approx [ (first_edge_concave_vertex_clockwise + j) % len ( approx ) ] [ 0 ] [ 0 ] ) , int ( approx [ (first_edge_concave_vertex_clockwise + j) % len ( approx ) ] [ 0 ] [ 1 ] )) , radius = 4 , color = (255 , 0 , 255) , thickness = -1 )

						# we can exit from the for of approx edges
						break

				#actual_image = cv2.putText ( actual_image , str ( i ) , (approx [ i ] [ 0 ] [ 0 ] , approx [ i ] [ 0 ] [ 1 ]) , cv2.FONT_HERSHEY_SIMPLEX , 1 , (0 , 0 , 255) , 2 )

				#print ( cv2.isContourConvex ( np.array ( [ np.array ( approx [ i ] ) , np.array ( approx [ (i + 1) % 5 ] ) , np.array ( approx [ (i + 2) % 5 ] ) ] ) ) )

				#approx [ 0 ] [ (i + 1) % len ( approx ) ]

				#print ( [ approx [ i ] [ 0 ] [ 0 ] , approx [ i ] [ 0 ] [ 1 ] ] , [ approx [ (i + 1) % len ( approx ) ] [ 0 ] [ 0 ] , approx [ (i + 1) % len ( approx ) ] [ 0 ] [ 1 ] ] )

				#a = math.dist ( [ approx [ i ] [ 0 ] [ 0 ] , approx [ i ] [ 0 ] [ 1 ] ] , [ approx [ (i + 1) % len ( approx ) ] [ 0 ] [ 0 ] , approx [ (i + 1) % len ( approx ) ] [ 0 ] [ 1 ] ] )
				#print ( np.array ( [ np.array ( approx [ i ] ) , np.array ( approx [ (i + 1) % 5 ] ) , np.array ( approx [ (i + 2) % 5 ] ) ] ) )
				#print ( "\n" )
				#print ( "-------------------------------------" )
				#print ( perimeter )
				#print ( "\n\n\n" )

				"""if cv2.isContourConvex(edge):
                    cv2.drawContours ( actual_image , [ edge ] , 0 , (255 , 0 , 0) , 5 )

                else:
                    cv2.drawContours ( actual_image , [ edge ] , 0 , (0 , 255 , 255) , 5 )"""

				#print ( hierarchy [ 0 ] [ i ] )

				# initialize the list of children of actual polygon as empty
				children_contours = [ ]

				# we know that contours have form [next, previous, first child, parent]
				children = hierarchy [ 0 ] [ index_actual_contour ] [ 2 ]

				# while there are other childrens
				while children != -1 :

					# append actual children to the previous list
					children_contours.append ( children )

					# obtain the right contour
					children = hierarchy [ 0 ] [ children ] [ 0 ]

				# area children contours
				area_children_contours = [ ]

				# for each children contours
				for children_contour in children_contours :

					# calculate area actual children
					area_actual_children = cv2.contourArea ( contours [ children_contour ] )

					# save the actual area
					area_children_contours.append ( area_actual_children )

				# temp array to store the children contours
				temp_children_contours = children_contours.copy ( )

				# initialize as empty array the children contours
				children_contours = [ ]

				# for each area contour
				for i in range ( 0 , len ( area_children_contours ) ) :

					# if the actual area contour is greater than 0 and so it is a valid area
					if area_children_contours [ i ] > 0 :

						# save the actual children contour as valid
						children_contours.append ( temp_children_contours [ i ] )

				# convert the children contours list in numpy array
				children_contours = np.array ( children_contours )

				# if there are children in actual contour
				if len ( children_contours ) > 0 :

					# we can draw contours of the actual region but in the colored image
					cv2.drawContours ( actual_image , [ contour ] , 0 , (0 , 0 , 255) , 1 )

				#----------------------------------------------------------------------------------------------------------------------------------------

				# if there are 5 children
				if len ( children_contours ) == 5 :

					# create a empty list to store all important points for each polygon
					points = [ ]

					cv2.drawContours ( actual_image , [ contour ] , 0 , (0 , 0 , 255) , 5 )

					# calculate the edge of the outer polygon
					second = approx [ (first_edge_concave_vertex_clockwise + 2) % len ( approx ) ] [ 0 ]
					third = approx [ (first_edge_concave_vertex_clockwise + 3) % len ( approx ) ] [ 0 ]

					# calculate the middle point of the previous edge
					half_second_third = [ (second [ 0 ] + third [ 0 ]) // 2 , (second [ 1 ] + third [ 1 ]) // 2 ]

					# calculate the middle point between the previous middle point and the concave vertex
					half_point_polygon = [ (approx [ first_edge_concave_vertex_clockwise ] [ 0 ] [ 0 ] + half_second_third [ 0 ]) // 2 , (approx [ first_edge_concave_vertex_clockwise ] [ 0 ] [ 1 ] + half_second_third [ 1 ]) // 2 ]

					# append to the list the outer point
					points.append ( half_second_third )

					# append to the list the middle point of the middle edge in the polygon
					points.append ( half_point_polygon )

					# append to the list the inner point
					points.append ( [ approx [ first_edge_concave_vertex_clockwise ] [ 0 ] [ 0 ] , approx [ first_edge_concave_vertex_clockwise ] [ 0 ] [ 1 ] ] )

					# for each children contour
					for single_children in children_contours :

						# draw contours of actual children
						cv2.drawContours ( actual_image , contours , single_children , (255 , 0 , 0) , 6 )

						# calculate the center of actual contour
						M = cv2.moments ( contours [ single_children ] )
						if M [ "m00" ] != 0 :
							cX = int ( M [ "m10" ] / M [ "m00" ] )
							cY = int ( M [ "m01" ] / M [ "m00" ] )

							# append to the list the central point of actual contour
							points.append ( [ cX , cY ] )

					# save the coordinates of the outer point
					x = points [ 0 ] [ 0 ]
					y = points [ 0 ] [ 1 ]

					# order list of points from outer to inner points through a line 2d
					points.sort ( key = lambda p : (p [ 0 ] - x) ** 2 + (p [ 1 ] - y) ** 2 )

					print ( image )

					print ( points )
					print ( len ( points ) )
					for i in range ( 1 , 5 ) :
						print ( points [ i ] , end = " - " )
						print ( points [ i + 1 ] , end = " - " )
						print ( math.dist ( points [ i ] , points [ i + 1 ] ) )

					print ( "\n\n" )

				#lis.sort(key=lambda x: (int(x[0]),int(x[1])))

				#----------------------------------------------------------------------------------------------------------------------------------------

				# for each children
				for single_children in children_contours :

					# draw contour
					cv2.drawContours ( actual_image , contours , single_children , (0 , 255 , 0) , 1 )

					M = cv2.moments ( contours [ single_children ] )
					if M [ "m00" ] != 0 :
						cX = int ( M [ "m10" ] / M [ "m00" ] )
						cY = int ( M [ "m01" ] / M [ "m00" ] )
						cv2.circle ( actual_image , (cX , cY) , 1 , (40 , 40 , 40) , -1 )

		#M = cv2.moments ( contour )
		#cX = int ( M [ "m10" ] / M [ "m00" ] )
		#cY = int ( M [ "m01" ] / M [ "m00" ] )
		# draw the center of the shape on the image
		# cv2.circle ( actual_image , (cX , cY) , 1 , (0 , 0 , 255) , -1 )

		# increment the index of actual contour
		index_actual_contour = index_actual_contour + 1

	"""i = 0
	# for each contour in threshold_gray_actual_image
	for contour in contours_threshold_gray_actual_image :

		# calculate the area of the actual region
		area_actual_region = cv2.contourArea ( contour )

		# approximate the shape of the actual polygon
		approx = cv2.approxPolyDP ( contour , 0.01 * cv2.arcLength ( contour , True ) , True )

		# if the area is acceptable
		if 200 <= area_actual_region <= 400 :

			if 6 <= len ( approx ) <= 20 :

				for j in list_black_polygon :

					if hierarchies [ j , i , 3 ] == -1 :

						# we can dra contours of the actual region but in the colored image
						cv2.drawContours ( actual_image , [ approx ] , 0 , (0 , 255 , 0) , 2 )

						print ( "cciao" )

		i = i + 1"""
	cv2.imwrite ( image.replace ( "uncalibrated" , "polygonborders" ) , actual_image )




framesVideo1/uncalibrated\10.jpg
[[1353, 39], [1349, 68], [1346, 93], [1343, 117], [1343, 118], [1340, 142], [1337, 166], [1333, 195]]
8
[1349, 68] - [1346, 93] - 25.179356624028344
[1346, 93] - [1343, 117] - 24.186773244895647
[1343, 117] - [1343, 118] - 1.0
[1343, 118] - [1340, 142] - 24.186773244895647



framesVideo1/uncalibrated\11.jpg
[[1416, 67], [1408, 95], [1401, 120], [1395, 144], [1395, 145], [1388, 169], [1382, 193], [1375, 221]]
8
[1408, 95] - [1401, 120] - 25.96150997149434
[1401, 120] - [1395, 144] - 24.73863375370596
[1395, 144] - [1395, 145] - 1.0
[1395, 145] - [1388, 169] - 25.0



framesVideo1/uncalibrated\12.jpg
[[1476, 144], [1464, 169], [1453, 191], [1444, 211], [1443, 213], [1433, 234], [1423, 255], [1412, 279]]
8
[1464, 169] - [1453, 191] - 24.596747752497688
[1453, 191] - [1444, 211] - 21.93171219946131
[1444, 211] - [1443, 213] - 2.23606797749979
[1443, 213] - [1433, 234] - 23.259406699226016



framesVideo1/uncalibrated\13.jpg
[[1530, 293], [1514, 310], [1500